# Prometheus & Grafana

Prometheus and Grafana form one of the most widely used open-source monitoring stack, especially for containerized environments such as K8s.

Briefly speaking, Prometheus is a tool that regularly scrapes metrics from targets, such as inference services in our use case of model serving. These targets (e.g., inference services) expose service-related metrics like the total number of requests and response latency in a specific format that you'll see later. Prometheus stores the collected metrics in a time-series database and provides a query language named PromQL to enable users to query, aggregate, and analyze the historical metrics. 

Grafana, complementing Prometheus, allows users to create interactive and customizable dashboards to visualize metrics collected by Prometheus. You'll see a concrete example of using Grafana later in this tutorial. 

In fact, you already deployed Prometheus and Grafana when you set up the MLOps platform at the beginning of the course. Before diving into the rest of the tutorial, please watch [this YouTube video](https://www.youtube.com/watch?v=h4Sl21AKiDg) to better understand how Prometheus works. 

**Note**: There are links to example videos of using Prometheus and Grafana in this notebook. The videos are silent but using subtitles to explain what's going on. 

# Prometheus & Grafana in the MLOps platform
As mentioned in the video that introduces Prometheus, the services that need to be monitored should expose their metrics through an HTTP endpoint. Prometheus can then pull these metrics from the exposed endpoint. Fortunately, the inference services deployed on KServe come with a built-in metric-exposing endpoint, so we don't need to implement one ourselves.

In particular, each pod running an inference service comprises two containers: an application container, which contains the runtime environment for serving the model, and a side-car container named "queue-proxy." These containers expose distinct service-level metrics. For the sake of simplicity, our course will concentrate on the metrics provided by the "queue-proxy" side-car container. If you'd like to delve deeper into the metrics exposed by the model-serving container, you can find additional information [here](https://kserve.github.io/website/0.10/modelserving/observability/prometheus_metrics/).


When we incorporate the monitoring stack of Prometheus and Grafana into our MLOps platform architecture, our diagram looks like the one below. In this setup, Prometheus retrieves metrics from the inference services running in KServe. Grafana can then be used to visualize the metrics collected by Prometheus. 

<img src="./images/overview-monitoring-focus.jpg" width=700>

Now, let's see how to use Prometheus and Grafana in the MLOps platform. 

First, let's make sure the the monitoring stack of Prometheus and Grafana is up.

In [1]:
!kubectl get pods -n monitoring

NAME                                     READY   STATUS    RESTARTS   AGE
grafana-9fb69f845-r5pqm                  1/1     Running   0          39d
prometheus-deployment-7b898cb9d8-7ggrl   1/1     Running   0          39d


You are expected to see two pods running, one for Grafana and one for the Prometheus main service:

```text
NAME                                     READY   STATUS    RESTARTS   AGE
grafana-9fb69f845-lhdrh                  1/1     Running   0          2d2h
prometheus-deployment-7b898cb9d8-n2s29   1/1     Running   0          2d2h
```

As mentioned above, inference services running in KServe already expose metrics which can be scraped by Prometheus. Let's see how to configure an inference service and Prometheus so that we can really check the metrics from Prometheus. In this example, we'll use the familiar model that predicts red wine quality. 

Let's first deploy a model for wine quality prediction. The model is the same one used in the MLflow tutorial of the first week. The deployed inference service is named "wine-quality". (Again, remember to replace the storageUri with your own one in [manifests/wine-quality-model.yaml](./manifests/wine-quality-model.yaml) if you want to run the tutorial.)

In [2]:
!kubectl apply -f manifests/wine-quality-model.yaml

inferenceservice.serving.kserve.io/wine-quality created


### Prometheus - metric scraping
Looking at [manifests/wine-quality-model.yaml](./manifests/wine-quality-model.yaml), you may find it quite similar to the configuration we used in previous weeks. The only difference is that now the inference service is annotated with one line:
```yaml
prometheus.io/scrape: "true"
```
To explain why we need such an annotation, we need to look at the configuration of the Prometheus service. The configuration has been deployed as a ConfigMap to the K8s cluster (named "prometheus-server-conf" under the "monitoring" namespace) and its specification can be found [here](https://version.helsinki.fi/luoyumo/engineering_of_ml_systems/-/blob/master/pre-materials/setup/install_platform/deployment/monitoring/prometheus/prometheus-config-map.yaml?ref_type=heads). This ConfigMap is then consumed by the pod running the Prometheus service.

Let's focus on the configuration of metric scraping:
```yaml
scrape_configs:
  - job_name: 'kserve-inference-service-pods'
    kubernetes_sd_configs:
    - role: pod     
    relabel_configs:
    # Configure Prometheus to scrape metrics from pods annotated with "prometheus.io/scrape"
    - source_labels: [__meta_kubernetes_pod_annotation_prometheus_io_scrape]
      action: keep
      regex: true
    # Besides being annotated with "prometheus.io/scrape",
    # the The pods to be scraped should also have a label whose key is "serving.kserve.io/inferenceservice.
    # Prometheus should scrape the metrics from a container port named "http-usermetric". Kserve inference services use this port to expose metrics.
    - source_labels: [__meta_kubernetes_pod_label_serving_kserve_io_inferenceservice,  __meta_kubernetes_pod_container_port_name]
      action: keep
      regex: (.+);http-usermetric
    # Extract the inference service (isvc) name from the pod name to more clearly show which metric is from which inference service
    - source_labels: [__meta_kubernetes_pod_name]
      separator: '-predictor-default-'
      regex: '(.*)-predictor-default-(.*)'
      replacement: '${1}'
      target_label: isvc_name
```
You don't need to fully understand the Prometheus configuration to complete the assignments. The configuration is shown here just to give you a general idea of how to configure Prometheus to scrape metrics from inference services. If you're interested at learning more about Prometheus configuration, you can refer to the [official documentation](https://prometheus.io/docs/prometheus/latest/configuration/configuration/).

In [3]:
# Make sure that the "wine-quality" inference service is ready
!kubectl -n kserve-inference get isvc wine-quality

NAME           URL                                                READY   PREV   LATEST   PREVROLLEDOUTREVISION   LATESTREADYREVISION                    AGE
wine-quality   http://wine-quality.kserve-inference.example.com   True           100                              wine-quality-predictor-default-00001   4m52s


Expected output:
```text
NAME           URL                                                READY   PREV   LATEST   PREVROLLEDOUTREVISION   LATESTREADYREVISION                    AGE
wine-quality   http://wine-quality.kserve-inference.example.com   True           100                              wine-quality-predictor-default-00009   19d
```

Let's look at how the exposed metrics are like. 

First, let's send some requests to the "wine-quality" inference service.


In [ ]:
# Send some requests
from utils import send_requests

# Send 30 requests in every 0.5s, each request has one input(i.e., data point)
send_requests(count=30, input_length=1)

Next, we need to open a tunnel between our local environment and the pod's endpoint exposing the metrics. (By default, the metrics are exposed at the "queue-proxy" container's port 9091 and this is the port named "http-usermetric" that you saw in the Prometheus configuration above.)
```bash
export pod_name=$(kubectl -n kserve-inference get pods -l serving.kserve.io/inferenceservice=wine-quality|grep "wine-quality"| cut -d' ' -f1)
kubectl -n kserve-inference port-forward pod/$pod_name 9091:9091
```
Then, navigate to [http://localhost:9091/metrics](http://localhost:9091/metrics) and you will see some metrics there. For example,

<img src="./images/endpoint-metrics.png"/>

**Note**: 
- The lifetime of these metrics is 10 minutes and they'll disappear from the "/metrics" endpoint after 10 minutes. 
- If you run this tutorial yourself, the exact values of the metrics may vary, depending on the number (and frequency) of the requests sent to your inference service. 

As mentioned in the YouTube video of Prometheus, Prometheus metrics have different types like Counter, Gauge, and Histogram. 

Let's first look at the "revision_app_request_count" metric:
```
revision_app_request_count{configuration_name="wine-quality-predictor-default",container_name="queue-proxy",namespace_name="kserve-inference",pod_name="wine-quality-predictor-default-00003-deployment-84b4f8d46fpfffm",response_code="200",response_code_class="2xx",revision_name="wine-quality-predictor-default-00003",service_name="wine-quality-predictor-default"} 30
```
This metric is a Counter. A Counter keeps track of the number of occurrences of a specific event. 
A Prometheus metric consists of three parts:
1. Metric name: In this example, the metric name is "revision_app_request_count." This name provides information about what the metric is measuring. In this case, the metric represents the number of requests routed to the pod serving the model. 
2. Labels: Labels are used to differentiate the characteristics of the  metric and provide additional context to the metric. For instance, the labels `namespace_name="kserve-inference," pod_name="wine-quality-predictor-default-00003-deployment-84b4f8d46fpfffm," response_code="200"` tell us that the metric measures the number of requests that 1) was routed to a pod named "wine-quality-predictor..." running in the "kserve-inference" namespace, 2)received a 200 status code response.
3. Metric value. The value is currently 30 because we just sent 30 requests to the "wine-quality" inference service. 

Next, let's move to the "revision_app_request_latencies" metric. This metric is a Histogram, which is used to track the distribution of values in a set of observations. It is particularly useful for understanding the statistical characteristics of a metric over a period od time, such as percentiles of values. A Histogram metric is a combination of multiple Counter metrics. As you can see, the "revision_app_request_latencies" metric are divided into buckets:
```
revision_app_request_latencies_bucket{configuration_name="wine-quality-predictor-default",container_name="queue-proxy",namespace_name="kserve-inference",pod_name="wine-quality-predictor-default-00003-deployment-84b4f8d46fpfffm",response_code="200",response_code_class="2xx",revision_name="wine-quality-predictor-default-00003",service_name="wine-quality-predictor-default",le="5"} 28
revision_app_request_latencies_bucket{configuration_name="wine-quality-predictor-default",container_name="queue-proxy",namespace_name="kserve-inference",pod_name="wine-quality-predictor-default-00003-deployment-84b4f8d46fpfffm",response_code="200",response_code_class="2xx",revision_name="wine-quality-predictor-default-00003",service_name="wine-quality-predictor-default",le="10"} 30
```
The `le` label indicates the upper bound of each bucket. For example, the first bucket `revision_app_request_latencies_bucket{...,le="5"} 28` means 28 requests received responses within 5ms, and the second bucket `revision_app_request_latencies_bucket{...,le="10"} 30` means all of the 30 requests received responses within 10ms. 

In the next step, let's see how to use PromQL to query the metrics and visualize them in Grafana. 

Let's again send some requests to the "wine-quality" inference service

In [ ]:
# Increase the number of inputs in each request to intentionally increase the response time. It'll take ~1min30s to finish this function
send_requests(count=30, input_length=100000)

Let's navigate to the Prometheus UI at [http://prometheus-server.local](http://prometheus-server.local). Prometheus scrapes the metrics every 15 seconds in our setup. This scraping interval can be configured. 

Let's first enter a metric name to the query field and we can see the familiar metrics appear:

<img src="./images/prom-basic-query.png" width=1200/>

Compared to the metrics we previously saw at the pod endpoint, there are two main differences:
1. The metrics appear in the Prometheus UI have a new label "isvc_name=wine-quality" since we configured Prometheus to extract the inference service name from the pod name when scraping the metrics:
```yaml
- source_labels: [__meta_kubernetes_pod_name]
    separator: '-predictor-default-'
    regex: '(.*)-predictor-default-(.*)'
    replacement: '${1}'
    target_label: isvc_name
```
We can use this label to filter the metrics related to a specific inference service if we have multiple inference services. E.g., `revision_app_request_latencies_bucket{isvc_name="wine-quality"}`.

2. Now, 30 requests received responses within 30ms and all of the 60 requests received responses within 1000ms because we sent 30 requests, each of which had many inputs. 

These buckets themselves may not directly give us the information we want. For example, we probably care about the 90th percentile of the response latency. This is where PromQL comes into the play. PromQL provides many built-in functions to help us analyze the metrics. For example, we can use the following query to check the 90th percentile of the response latency of the "wine-quality" inference service in the last 5 minutes. In other words, we want to know the latency value below which 90% of the requests fall.

``` 
histogram_quantile(0.9, rate(revision_app_request_latencies_bucket{isvc_name="wine-quality"}[5m]))
```

<img src="./images/90-percentile-latency.png" width=1000>

From the screenshot, we can see that the 90th percentile of the response latency is around 1833ms, which makes sense as half of the requests have a large number of inputs. 

Let's look at the query we just used. 
- `revision_app_request_latencies_bucket{isvc_name="wine-quality"}`: This part selects the "revision_app_request_latencies_bucket" Histogram metric coming from the pod running for the "wine-quality" inference service.
- `rate(...)`: This function calculates the per-second average increase rate for each bucket.
- `[5m]`: This specifies a time range of the past 5 minutes, indicating that we want to calculate the increase rate of the buckets over the last 5 minutes.
- `histogram_quantile(0.9, ...)`: Finally, given the increase rate of each bucket of the metric, the histogram_quantile function is used to calculate the percentile of a Histogram. In this case, it calculates the 90th percentile (0.9 quantile) of the rate of request latencies for the "wine-quality" service over the past 5 minutes. 

**Note**: The result is an approximation because the "histogram_quantile" function interpolates quantile values by assuming a linear distribution within a bucket.

If you're interested, you can learn more about PromQL from [Prometheus documentation](https://prometheus.io/docs/prometheus/latest/querying/basics/).

### Prometheus - alerting

Prometheus provides an alerting feature to notify abnormal metrics. For example, we can configure Prometheus to send an alert if the 90th percentile response latency is too large so that the operation team can step in to further investigate the underlying reason and fix the problem. 

To use the alerting feature, we need to configure alerting rules for Prometheus. The alerting configuration is specified [in the same file](https://version.helsinki.fi/luoyumo/engineering_of_ml_systems/-/blob/master/pre-materials/setup/install_platform/deployment/monitoring/prometheus/prometheus-config-map.yaml?ref_type=heads) as the scraping configuration.
```yaml
groups:
    - name: Prediction latency alert
      rules:
      - alert: High prediction latency
        expr: histogram_quantile(0.9, rate(revision_app_request_latencies_bucket{isvc_name="wine-quality"}[5m])) > 500
        for: 5s
        labels:
            severity: warning
        annotations:
            summary: High prediction latency
```
The "expr" field specifies the PromQL query used to retrieve a metric and define an alert condition. In this particular case, our query focuses on the 90th percentile of response latency for the "wine-quality" inference service. We instruct Prometheus to trigger an alert if the 90th percentile of response latency exceeds 500ms.

Initially, when the condition of high latency is met, the alert will be in a pending state. If this condition persists for a duration of 5 seconds (as defined in the "for" field), Prometheus will then trigger the alert. Once the alert is fired, it can be directed to various downstream channels such as Slack and email for notification (This requires additional configuration and isn't covered in this tutorial). In our setup, Prometheus evaluates the metric against the alerting rule set in the "expr" field every 15s. Similar to the scraping interval, this evaluating interval is also configurable.

[This video](https://www.youtube.com/watch?v=MiJEjXyu12Q) shows an example of Prometheus alerting when the 90th percentile of the response time of the "wine-quality" inference service is too high. 

Some clarifications of the video:
- You need to refresh the Prometheus UI page to see the updates status of an alert.
- A triggered alert will become inactive if the alert condition is not met when Prometheus runs the PromQL query defined in the "expr" field once again.

For more detailed information on configuring alerts in Prometheus, you can refer to the [Prometheus documentation](https://prometheus.io/docs/alerting/latest/configuration/#configuration). 

### Grafana - metric visualization
(This is a complementary part. You don't need to use Grafana in the assignments.)

Now, we know how to query and analyze the metrics in Prometheus. The next step can be visualizing them in Grafana.

From a high-level perspective, Grafana enables its users to create dashboards, which serve as customizable visualization interfaces for monitoring and analyzing data. Each dashboard can contain one or more panels, and these panels are the basic blocks for visualizing data. Consider the example below of a dashboard designed for monitoring application performance. This dashboard includes multiple panels, each serving a specific purpose. For instance, there is one panel for visualizing memory and CPU usage, another for tracking incoming requests to individual servers, and so on.

<figure>
    <img src="./images/grafana-dashboard-example.png" width=1000 />
    <figcaption>
    <i>Source: https://grafana.com/oss/grafana/</i>
  </figcaption>
</figure>

We can access the Grafana UI at [http://grafana-server.local](http://grafana-server.local). A username and password may be required to log in to Grafana UI. Both of them are "***admin***" in our setup. (Grafana may suggest you to change the username and password after the login but you can ignore it for now.)

[This video](https://youtu.be/l8_4j4Ik7yY) illustrates how to visualize the 90th percentile of the response time in the past 5 minutes of the "wine-quality" inference service in Grafana. 